<a href="https://colab.research.google.com/github/Cham0703/DU_AN_SPAM/blob/main/main_email_detechtion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Code 1

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import chardet
from wordcloud import WordCloud
import re
import nltk
from nltk.corpus import stopwords
# 🧾 BƯỚC 1: Đọc file từ ổ đĩa
duong_dan = "D:/spam.csv"
# Tự động phát hiện encoding của file để tránh lỗi khi đọc (máy tui bị lỗi f8 nên tui chạy chuẩn đoán cho chắc )
with open(duong_dan, "rb") as f:
    result = chardet.detect(f.read(10000))  # Đọc thử 10.000 byte
    encoding = result['encoding']
    print(f"Đã đoán encoding: {encoding}")
# Đọc dữ liệu và chuẩn hóa cột
df = pd.read_csv(duong_dan, encoding=encoding)
df = df[['v1', 'v2']]                    # Lấy 2 cột cần thiết
df.columns = ['label', 'message']       # Đặt tên cột rõ ràng
# 📊 BƯỚC 2: Khám phá dữ liệu (EDA)
# Thống kê số lượng ham/spam
print("Số lượng ham/spam:")
print(df['label'].value_counts())
# Vẽ biểu đồ cột số lượng
sns.countplot(data=df, x='label')
plt.title("Số lượng tin nhắn ham/spam")
plt.show()
# Tính độ dài tin nhắn
df['message_length'] = df['message'].apply(len)
# Vẽ biểu đồ phân bố độ dài tin nhắn
sns.histplot(data=df, x='message_length', hue='label', bins=50, kde=True)
plt.title("Độ dài tin nhắn theo nhãn")
plt.show()
# ☁️ BƯỚC 3: Tạo WordCloud
spam_text = ' '.join(df[df['label'] == 'spam']['message'])
ham_text = ' '.join(df[df['label'] == 'ham']['message'])
plt.figure(figsize=(12,6))
plt.subplot(1, 2, 1)
plt.imshow(WordCloud(width=500, height=300, background_color='white').generate(spam_text))
plt.axis("off")
plt.title("Spam WordCloud")
plt.subplot(1, 2, 2)
plt.imshow(WordCloud(width=500, height=300, background_color='white').generate(ham_text))
plt.axis("off")
plt.title("Ham WordCloud")
plt.show()
# 🧹 BƯỚC 4: Làm sạch tin nhắn
# Đường dẫn đến file spam.csv của bạn
duong_dan = "D:/spam.csv"
# Phát hiện encoding để tránh lỗi Unicode
with open(duong_dan, "rb") as f:
    encoding = chardet.detect(f.read())['encoding']
# Đọc dữ liệu và đổi tên cột
df = pd.read_csv(duong_dan, encoding=encoding)[['v1', 'v2']]
df.columns = ['label', 'message']
# Tải stopwords nếu chưa có
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
# Hàm làm sạch văn bản
def clean_message(msg):
    msg = msg.lower()                              # Chuyển sang chữ thường
    msg = re.sub(r'\d+', '', msg)                 # Xoá chữ số
    msg = re.sub(r'[^\w\s]', '', msg)             # Xoá dấu câu
    words = msg.split()                           # Tách từ
    words = [w for w in words if w not in stop_words]  # Bỏ stop words
    return ' '.join(words)
# Tạo cột mới chứa tin nhắn đã làm sạch
df['cleaned_message'] = df['message'].apply(clean_message)
# Chỉ giữ lại hai cột yêu cầu: nhãn và tin nhắn đã làm sạch
df_output = df[['label', 'cleaned_message']]
# Hiển thị 5 dòng đầu để kiểm tra
print(df_output.head())

FileNotFoundError: [Errno 2] No such file or directory: 'D:/spam.csv'

Code 2

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

duong_dan = "D:/spam.csv"

# BƯỚC 1: Chuyển đổi nhãn sang dạng số
# Chuyển 'ham' thành 0, 'spam' thành 1
df['label_num'] = df['label'].map({'ham': 0, 'spam': 1})

# In ra để kiểm tra
print("Số lượng mẫu theo nhãn số:")
print(df['label_num'].value_counts())

# BƯỚC 2: Chuyển đổi tin nhắn thành đặc trưng số
# Sử dụng CountVectorizer (Mô hình Bag of Words)
count_vectorizer = CountVectorizer(max_features=5000)  # Giới hạn 5000 từ phổ biến nhất
X_count = count_vectorizer.fit_transform(df['cleaned_message'])

# Sử dụng TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer.fit_transform(df['cleaned_message'])

# In ra kích thước của dữ liệu sau khi chuyển đổi
print(f"Kích thước dữ liệu CountVectorizer: {X_count.shape}")
print(f"Kích thước dữ liệu TF-IDF: {X_tfidf.shape}")

# BƯỚC 3: Chia dữ liệu thành tập huấn luyện và tập kiểm tra
# Sử dụng cả hai loại đặc trưng để so sánh
# Sử dụng CountVectorizer
X_count_train, X_count_test, y_count_train, y_count_test = train_test_split(
    X_count, df['label_num'], test_size=0.2, random_state=42, stratify=df['label_num']
)

# Sử dụng TF-IDF
X_tfidf_train, X_tfidf_test, y_tfidf_train, y_tfidf_test = train_test_split(
    X_tfidf, df['label_num'], test_size=0.2, random_state=42, stratify=df['label_num']
)

# BƯỚC 4: Lưu từ vựng để sử dụng sau này
count_vocab = count_vectorizer.get_feature_names_out()
tfidf_vocab = tfidf_vectorizer.get_feature_names_out()

# In ra thông tin về tập huấn luyện và kiểm tra
print("\nThông tin về bộ dữ liệu đã chia:")
print(f"Số lượng mẫu huấn luyện: {X_count_train.shape[0]}")
print(f"Số lượng mẫu kiểm tra: {X_count_test.shape[0]}")
print(f"Tỷ lệ spam trong tập huấn luyện: {y_count_train.mean()*100:.2f}%")
print(f"Tỷ lệ spam trong tập kiểm tra: {y_count_test.mean()*100:.2f}%")


output_summary = {
    'CountVectorizer': {
        'X_train shape': X_count_train.shape,
        'X_test shape': X_count_test.shape,
        'y_train shape': y_count_train.shape,
        'y_test shape': y_count_test.shape,
        'vocabulary size': len(count_vocab)
    },
    'TfidfVectorizer': {
        'X_train shape': X_tfidf_train.shape,
        'X_test shape': X_tfidf_test.shape,
        'y_train shape': y_tfidf_train.shape,
        'y_test shape': y_tfidf_test.shape,
        'vocabulary size': len(tfidf_vocab)
    }
}

print("\nTổng kết:")
for method, details in output_summary.items():
    print(f"\n{method}:")
    for key, value in details.items():
        print(f"  - {key}: {value}")

NameError: name 'df' is not defined